In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.config.cfg_loader import env_setting
from lib.utils.base_funtion import build_dataloaders

cfg = env_setting(cfg_name='plscore_pureRMT_sigma_center.yaml')
loader_train, loader_val = build_dataloaders(cfg)
# data = None
# for i, _data in enumerate(loader_train, 1):
#     data = _data
#     break

In [3]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.plscore_RMT_sigma import PLScoreRMT
from lib.models.LeeNet.score_RMT_sigma_center import ScorePureRMTCENTER
from lib.models.layer.RMT import PatchMerging
from lib.models.head.center_predictor import CenterPredictor
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import get_optimizer_scheduler

from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PLScoreRMT(down_sample=PatchMerging, cfg=cfg)

    stride = cfg.model.backbone.stride
    feat_sz = int(cfg.data.search.size / stride)
    head = CenterPredictor(inplanes=cfg.model.pureRMT.embed_dim[-1], channel=cfg.model.head.num_channels, feat_sz=feat_sz, stride=stride)
    model = ScorePureRMTCENTER(backbone, head, cfg)
    return model




/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# net = build_model(cfg)
# 
# focal_loss = FocalLoss()
# objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
# loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
# actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

In [5]:
# data = data.to(cfg.train.device)
# actor(data)

In [6]:
# checkpoint_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/LeeNet_plScore_RMT_CENTER00001/ScorePureRMTCENTER_ep2600.pth.tar"
# # checkpoint_dict = torch.load(checkpoint_path,map_location={"cuda:1":"cuda:1"})['net']
# checkpoint_dict = torch.load(checkpoint_path)['net']
# 
# net = build_model(cfg)
# 
# model_dict = net.state_dict()
# 
# state_dict = {k: v for k, v in checkpoint_dict.items() if k in model_dict.keys()}
# need_delete_keys = []
# del state_dict['mlp.layers.0.weight']
# for key in state_dict.keys():
#     if 'backbone' in key or 'gen_feat_conv' in key or 'conv1_ctr' in key or 'conv1_offset' in key or 'conv1_size' in key:
#         print("delete key: ", key)
#         need_delete_keys.append(key)
# 
# print(need_delete_keys)
# 
# for key in need_delete_keys:
#     del state_dict[key]
# print(state_dict.keys())
# model_dict.update(state_dict)
# net.load_state_dict(model_dict, strict=False)

In [7]:
from collections import OrderedDict

RMT_L_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/RMT-L.pth"

RMT_L_dict = torch.load(RMT_L_path)['model']

net = build_model(cfg)
model_dict = net.state_dict()

RMT_state_dict = OrderedDict()

for key, v in RMT_L_dict.items():
    if key.startswith("layers"):
        key = key.replace("layers", "backbone.RMT_layers")
        RMT_state_dict[key] = v

# state_dict = {k: v for k, v in RMT_state_dict.items() if k in model_dict.keys()}
# for key in state_dict.keys():
#     print(key)
model_dict.update(RMT_state_dict)
net.load_state_dict(model_dict, strict=False)

<All keys matched successfully>

In [8]:
import torch.backends.cudnn
torch.backends.cudnn.benchmark = True

focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

trainer.train(cfg.train.epoch)

Epoch Time: 0:00:23.122539
Avg Data Time: 10.23540
Avg GPU Trans Time: 0.09577
Avg Forward Time: 12.79137
Epoch Time: 0:00:25.811710
Avg Data Time: 5.15441
Avg GPU Trans Time: 0.09569
Avg Forward Time: 7.65575
Epoch Time: 0:00:27.937003
Avg Data Time: 3.45431
Avg GPU Trans Time: 0.09578
Avg Forward Time: 5.76225
Epoch Time: 0:00:30.086493
Avg Data Time: 2.60608
Avg GPU Trans Time: 0.09499
Avg Forward Time: 4.82056
Epoch Time: 0:00:32.240869
Avg Data Time: 2.10069
Avg GPU Trans Time: 0.09545
Avg Forward Time: 4.25203
Epoch Time: 0:00:34.378559
Avg Data Time: 1.76015
Avg GPU Trans Time: 0.09548
Avg Forward Time: 3.87412
Epoch Time: 0:00:36.528442
Avg Data Time: 1.51836
Avg GPU Trans Time: 0.09607
Avg Forward Time: 3.60391
Epoch Time: 0:00:38.687449
Avg Data Time: 1.33599
Avg GPU Trans Time: 0.09605
Avg Forward Time: 3.40390
Epoch Time: 0:00:40.825438
Avg Data Time: 1.19349
Avg GPU Trans Time: 0.09596
Avg Forward Time: 3.24671
[train: 1, 10 / 625] FPS: 22.3 (44.8)  ,  DataTime: 1.079 (0.0

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efa976791f0>
Traceback (most recent call last):
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/selectors.py", line 415, in select
 

Training crashed at epoch 2


KeyboardInterrupt: 